## 使用 intfloat/multilingual-e5-small 来做 embeddings

因为这里是 embeddings， 所以输出是一串数字。

In [6]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
print(embeddings)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())


tensor([[ 0.0422, -0.0388, -0.0573,  ...,  0.0257,  0.0308,  0.0497],
        [ 0.0534, -0.0153, -0.0367,  ...,  0.0775,  0.0623,  0.0112],
        [-0.0258, -0.0404, -0.0440,  ...,  0.0651,  0.0467,  0.0581],
        [ 0.0194, -0.0025, -0.0375,  ...,  0.0613,  0.0775,  0.0191]],
       grad_fn=<DivBackward0>)
[[92.72808837890625, 75.96248626708984], [74.3805160522461, 90.60895538330078]]


## VectorDB

In [2]:
import chromadb
chroma_client = chromadb.HttpClient(host='localhost', port=8000)

### Create a collection

In [3]:
collection = chroma_client.create_collection(name='intfloat_collection')

In [10]:
collection.add(
    embeddings=embeddings.tolist(),
    documents=["This is query 1", "This is query 2", "This is passage 1", "This is passage 2"],
    ids=["id1", "id2", "id3", "id4"],
)


### Query result

In [12]:
results = collection.query(
    query_texts=["This is a passage"],
    n_results=2
)
results

{'ids': [['id3', 'id1']],
 'distances': [[2.0443079705844664, 2.0569650703886198]],
 'embeddings': None,
 'metadatas': [[None, None]],
 'documents': [['This is passage 1', 'This is query 1']],
 'uris': None,
 'data': None}